same as two_scales_video_7.ipynb, but displays reading on a separate screen, so as to avoid some unnecessary computation.

still cannot eliminate the usage of min and max altogether, as they are needed for d1_value and d2_value calculations in get_value() functions, and also the radii of the concentric ellipses.

working model, displays but cannot save the file well

need to devise a method to draw concentric ellipse even if maximum or minimum isn't identified (using fractions of needle_length)

In [ ]:
import time
import csv
from matplotlib import pyplot as plt
import pandas as pd
from ultralytics import YOLO
import os
import cv2
import numpy as np
import pytesseract
import easyocr
import math
from paddleocr import PaddleOCR
import gc


def extract_elliptical_ring(image, center, axes_big, axes_small, angle_deg):
    # Create black mask
    mask = np.zeros(image.shape[:2], dtype=np.uint8)

    # Draw filled large ellipse (white)
    cv2.ellipse(mask, (center, axes_big, angle_deg), color=255, thickness=-1)

    # Draw filled small ellipse (black) inside to create ring
    cv2.ellipse(mask, (center, axes_small, angle_deg), color=0, thickness=-1)

    # Apply mask to image
    segmented = cv2.bitwise_and(image, image, mask=mask)

    # save_path = os.path.join(output_folder, os.path.basename(image_path))
    # Optional: Save segmented image
    # cv2.imwrite(save_path, segmented)

    return segmented


def detect_and_draw_boxes(image, ocr):
    recognized_numbers = []  # To store (position, integer value) tuples

    # Run OCR
    result = ocr.ocr(image)

    for block in result:
        if block is not None:
            for line in block:
                if line is not None:
                    box = line[0]
                    text = line[1][0]

                    pts = []
                    for point in box:
                        x, y = point
                        pts.append((int(x), int(y)))
                    pts_array = np.array(pts, np.int32)

                    # # Draw the bounding box
                    # cv2.polylines(image, [pts_array], isClosed=True, color=(0, 255, 0), thickness=2)

                    # # Put text above the bounding box
                    # cv2.putText(image, text, (pts[0][0], max(pts[0][1] - 10, 0)),
                    #             cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

                    # Compute center of bounding box
                    cx = int(sum(p[0] for p in pts) / 4)
                    cy = int(sum(p[1] for p in pts) / 4)

                    # Try parsing text as a clean integer
                    try:
                        stripped_text = text.strip()
                        value = float(stripped_text)  # Accepts decimals and integers

                        # # Draw red dot at center
                        # cv2.circle(image, (cx, cy), radius=4, color=(255, 0, 0), thickness=-1)

                        # Append to list
                        recognized_numbers.append(((cx, cy), value))
                    except ValueError:
                        pass  # Skip if not a number


    # save_path = os.path.join(output_folder, os.path.basename(image_path))
    # Save OCR result
    # cv2.imwrite(save_path, image)

    # return image, recognized_numbers
    return recognized_numbers



# Coordinate transformation functions
def rotate_point_1(x, y, cx, cy, angle_rad):
    x_shifted = x - cx
    y_shifted = y - cy
    x_rot = x_shifted * np.cos(angle_rad) + y_shifted * np.sin(angle_rad)
    y_rot = -x_shifted * np.sin(angle_rad) + y_shifted * np.cos(angle_rad)
    return x_rot, y_rot

def get_scaled_axes(X, Y, a_0, b_0):
    k = a_0 / b_0
    a_1 = np.sqrt(X**2 + ((k**2) * Y**2))
    b_1 = a_1 / k
    return a_1, b_1

def inverse_rotate_point_1(X, Y, cx, cy, angle_rad):
    x_unrot = X * np.cos(angle_rad) - Y * np.sin(angle_rad)
    y_unrot = X * np.sin(angle_rad) + Y * np.cos(angle_rad)
    return x_unrot + cx, y_unrot + cy

def draw_ellipse(image, cx, cy, a, b, angle_deg, color):
    angle_rad = np.deg2rad(angle_deg)
    ellipse_pts = []
    for t in np.linspace(0, 2 * np.pi, 500):
        X = a * np.cos(t)
        Y = b * np.sin(t)
        x, y = inverse_rotate_point_1(X, Y, cx, cy, angle_rad)
        pt = (int(x), int(y))
        if 0 <= pt[0] < image.shape[1] and 0 <= pt[1] < image.shape[0]:
            ellipse_pts.append(pt)

    for pt in ellipse_pts:
        cv2.circle(image, pt, 1, color, -1)


# # def ocr_detect(reader, idx, x_center, y_center, radius, ocr_regions_folder_1, image):
# def ocr_detect(reader, ocr, idx, x_center, y_center, radius, image):

#     # (x_center, y_center, radius)=circular_region

#     # Ensure coordinates and radius are integers and within bounds
#     x_center, y_center, radius = int(x_center), int(y_center), int(radius)

#     # Create a black mask with a white-filled circle
#     mask = np.zeros_like(image, dtype=np.uint8)
#     cv2.circle(mask, (x_center, y_center), radius, (255, 255, 255), thickness=-1)

#     # Apply the mask to extract the circular region
#     circular_region = cv2.bitwise_and(image, image, mask=mask[:, :, 0])
#     denoised = cv2.GaussianBlur(circular_region, (3,3), 0)

#     # # Save or visualize the edge-detected circular region
#     # denoised_path = os.path.join(ocr_regions_folder_1, f"denoised_circle_{idx}_{os.path.basename(image_path)}")
#     # cv2.imwrite(denoised_path, circular_region)
#     # print(f"Saved denoised circular region {idx} to: {denoised_path}")

#     result = ocr.ocr(denoised, cls=True)
#     # detected_number = None

#     # Extract the detected text (integer)
#     if result and result[0]:  # Ensure result is not empty
#         try:
#             return float(result[0][0][1][0])
#         except ValueError:
#             print(f"PaddleOCR could not convert the text to integer: {result[0][0][1][0]}")

#     value=""
#     results = reader.readtext(denoised)
#     for (bbox, text, confidence) in results:
#         value+=text

#     try:
#         return float(value)  # Attempt to convert the text to an integer
#     except ValueError:
#         print(f"EasyOCR could not convert the text to integer: {value}")

#     return None

# calculate clockwise angle from negative y axis


# Adjust angle calculation for elliptical gauge distortion
def rotate_point(point, angle_of_inclination):
    # Apply rotation matrix to the point
    rotation_matrix = np.array([
        [math.cos(angle_of_inclination), -math.sin(angle_of_inclination)],
        [math.sin(angle_of_inclination), math.cos(angle_of_inclination)]
    ])
    rotated_point = np.dot(rotation_matrix, point)
    return rotated_point

def stretch_point(point, major_axis_length, minor_axis_length):
    # Stretch the point based on ellipse's axes
    stretch_factor = major_axis_length / minor_axis_length
    stretched_point = np.array([point[0], point[1] * stretch_factor])
    return stretched_point

# def get_angle(point1, point2, point3, major_axis_length, minor_axis_length, ellipse_inclination):
def get_angle(point_1, point_2, major_axis_length, minor_axis_length, ellipse_inclination):
    """
    Compute the counterclockwise angle (in radians) from the positive x-axis between two points,
    then apply an elliptical correction factor based on the ratio of major_axis_length to minor_axis_length.
    
    Parameters:
      point1, point2: tuples (x, y) representing the two points.
      major_axis_length: the length of the ellipse's major axis.
      minor_axis_length: the length of the ellipse's minor axis.
    
    Returns:
      corrected_angle: The corrected angle (in radians) measured counterclockwise from the positive x-axis.
    """
    if point_1 is None or point_2 is None:
        return None  # Return None if any point is missing

    # Convert the angle from degrees to radians and invert it for a clockwise rotation.
    adjusted_inclination = -math.radians(ellipse_inclination)
    rotated_point_1 = rotate_point(point_1, adjusted_inclination)
    rotated_point_2 = rotate_point(point_2, adjusted_inclination)
    
    # Stretch the points based on ellipse axes (accounting for elliptical distortion)
    stretched_point1 = stretch_point(rotated_point_1, major_axis_length, minor_axis_length)
    stretched_point2 = stretch_point(rotated_point_2, major_axis_length, minor_axis_length)

    x1, y1 = point_1
    x2, y2 = point_2

    # Calculate the raw angle from the positive x-axis (counterclockwise), normalized to [0, 2*pi].
    raw_angle = (math.atan2(y2 - y1, x2 - x1) + 2 * math.pi) % (2 * math.pi)
    
    return raw_angle


def get_values_11(max_angle, min_angle, pointer_angle):
    
    print(f"Passed pointer_angle = {math.degrees(pointer_angle) if pointer_angle is not None else 'N/A'}, "
      f"max_angle = {math.degrees(max_angle) if max_angle is not None else 'N/A'}, "
      f"min_angle = {math.degrees(min_angle) if min_angle is not None else 'N/A'} to get_values_1()")

    """
    Given:
      - min_angle: the angle (in radians) of the minimum reading.
      - max_angle: the angle (in radians) of the maximum reading.
      - pointer_angle: the angle (in radians) of the needle pointer.
    
    Assumes that, in the "correct" gauge orientation, the minimum lies to the left and the maximum to the right.
    
    There are four general cases (with some duplicates in the original logic):
    
    (i)  If min_angle < max_angle < pointer_angle:
         The obtuse sector has the scale and the needle moves in clockwise direction,
         with the reference line lying in the scaled sector and the pointer below the reference.
         => angle_range = 2π - (max_angle - min_angle)
         => val_angle   = 2π - (pointer_angle - min_angle)
    
    (ii) If pointer_angle < min_angle < max_angle:
         The obtuse sector has the scale and the needle moves in clockwise direction,
         with the reference line in the scaled sector and the pointer above the reference.
         => angle_range = 2π - (max_angle - min_angle)
         => val_angle   = min_angle - pointer_angle
         
    (iii) If max_angle < pointer_angle < min_angle:
         The obtuse sector has the scale, the reference line lies outside the scaled sector,
         and the needle moves in clockwise direction.
         => angle_range = min_angle - max_angle
         => val_angle   = min_angle - pointer_angle
         
    (iv) If min_angle < pointer_angle < max_angle:
         The acute sector has the scale, the reference line lies outside the scaled sector,
         and the needle moves in counterclockwise direction.
         => angle_range = max_angle - min_angle
         => val_angle   = pointer_angle - min_angle
         
    (v)  If pointer_angle < max_angle < min_angle:
         (This case covers the situation where the acute sector is used, the reference line is in the scaled sector,
         and the needle moves in counterclockwise direction with the pointer above (or below) the reference.)
         Here we adjust pointer_angle:
         => angle_range = 2π - (min_angle - max_angle)
         => pointer_angle_adj = 2π - (min_angle - pointer_angle)
         => val_angle = pointer_angle_adj

    Returns:
      angle_range: the effective angular range of the scale (in radians)
      val_angle: the pointer's relative angle (in radians) within that range.
    """
    if min_angle < max_angle < pointer_angle:
        # Case (i)
        angle_range = 2 * math.pi - (max_angle - min_angle)
        val_angle = 2 * math.pi - (pointer_angle - min_angle)
        print(f'case (i) : ')
    elif pointer_angle < min_angle < max_angle:
        # Case (ii) -- covers duplicate branch
        angle_range = 2 * math.pi - (max_angle - min_angle)
        val_angle = min_angle - pointer_angle
        print(f'case (ii) : ')
    elif max_angle < pointer_angle < min_angle:
        # Case (iii)
        angle_range = min_angle - max_angle
        val_angle = min_angle - pointer_angle
        print(f'case (iii) : ')
    elif min_angle < pointer_angle < max_angle:
        # Case (iv)
        angle_range = max_angle - min_angle
        val_angle = pointer_angle - min_angle
        print(f'case (iv) : ')
    elif pointer_angle < max_angle < min_angle:
        # Case (v)
        angle_range = 2 * math.pi - (min_angle - max_angle)
        val_angle = 2 * math.pi - (min_angle - pointer_angle)
        print(f'case (v) : ')
    elif max_angle < min_angle < pointer_angle:
        # Case (vi)
        angle_range = 2 * math.pi - (min_angle - max_angle)
        val_angle = pointer_angle - min_angle
        print(f'case (vi) : ')
    else:
        # If none of the above conditions hold, return None or raise an error.
        angle_range = None
        val_angle = None
        print(f'case (vii) : ')
    
    print(f'angle_range = {math.degrees(angle_range)}')
    print(f'val_angle = {math.degrees(val_angle)}')
    
    return angle_range, val_angle


def get_values(max_angle, min_angle, pointer_angle, d1_angle, d2_angle, d1_value, d2_value):
    """
    This wrapper function selects the appropriate method to compute the effective
    angular range (angle_range) and pointer's relative angle (val_angle) based on the positions 
    of the major divisions relative to the gauge's min and max readings.
    
    Rules:
      - If d1_angle equals min_angle and d2_angle equals max_angle:
            call get_values_1(max_angle, min_angle, pointer_angle)
      - Else if d1_angle equals min_angle and d2_angle does not equal max_angle:
            call get_values_3(max_angle, min_angle, pointer_angle, d1_angle, d2_angle)
      - Else if d1_angle does not equal min_angle and d2_angle does not equal max_angle:
            call get_values_2(max_angle, min_angle, pointer_angle, d1_angle, d2_angle)
    """
    # Assume d1_angle and d2_angle are provided as parameters.
    if (d1_angle == min_angle) and (d2_angle == max_angle):
        print(f"Calling get_values_1(), passing values {d1_value} and {d2_value}")
        return get_values_1(max_angle, min_angle, pointer_angle)
    elif (d1_angle == min_angle) and (d2_angle != max_angle):
        print(f"Calling get_values_2(), passing values {d1_value} and {d2_value}")
        return get_values_2(max_angle, min_angle, pointer_angle, d1_angle, d2_angle)
    elif (d1_angle != min_angle) and (d2_angle == max_angle):
        print(f"Calling get_values_3(), passing values {d1_value} and {d2_value}")
        return get_values_3(max_angle, min_angle, pointer_angle, d1_angle, d2_angle)
    elif (d1_angle != min_angle) and (d2_angle != max_angle):
        print(f"Calling get_values_4(), passing values {d1_value} and {d2_value}")
        return get_values_4(max_angle, min_angle, pointer_angle, d1_angle, d2_angle)
    else:
        print("No matching case found in get_values()")
        return None, None



def get_values_1(max_angle, min_angle, pointer_angle):
    
    print(f"Passed pointer_angle = {math.degrees(pointer_angle) if pointer_angle is not None else 'N/A'}, "
      f"max_angle = {math.degrees(max_angle) if max_angle is not None else 'N/A'}, "
      f"min_angle = {math.degrees(min_angle) if min_angle is not None else 'N/A'} to get_values_1()")

    """
    Given:
      - min_angle: the angle (in radians) of the minimum reading.
      - max_angle: the angle (in radians) of the maximum reading.
      - pointer_angle: the angle (in radians) of the needle pointer.
    
    Assumes that, in the "correct" gauge orientation, the minimum lies to the left and the maximum to the right.
    
    There are four general cases (with some duplicates in the original logic):
    
    (i)  If min_angle < max_angle < pointer_angle:
         The obtuse sector has the scale and the needle moves in clockwise direction,
         with the reference line lying in the scaled sector and the pointer below the reference.
         => angle_range = 2π - (max_angle - min_angle)
         => val_angle   = 2π - (pointer_angle - min_angle)
    
    (ii) If pointer_angle < min_angle < max_angle:
         The obtuse sector has the scale and the needle moves in clockwise direction,
         with the reference line in the scaled sector and the pointer above the reference.
         => angle_range = 2π - (max_angle - min_angle)
         => val_angle   = min_angle - pointer_angle
         
    (iii) If max_angle < pointer_angle < min_angle:
         The obtuse sector has the scale, the reference line lies outside the scaled sector,
         and the needle moves in clockwise direction.
         => angle_range = min_angle - max_angle
         => val_angle   = min_angle - pointer_angle
         
    (iv) If min_angle < pointer_angle < max_angle:
         The acute sector has the scale, the reference line lies outside the scaled sector,
         and the needle moves in counterclockwise direction.
         => angle_range = max_angle - min_angle
         => val_angle   = pointer_angle - min_angle
         
    (v)  If pointer_angle < max_angle < min_angle:
         (This case covers the situation where the acute sector is used, the reference line is in the scaled sector,
         and the needle moves in counterclockwise direction with the pointer above (or below) the reference.)
         Here we adjust pointer_angle:
         => angle_range = 2π - (min_angle - max_angle)
         => pointer_angle_adj = 2π - (min_angle - pointer_angle)
         => val_angle = pointer_angle_adj

    Returns:
      angle_range: the effective angular range of the scale (in radians)
      val_angle: the pointer's relative angle (in radians) within that range.
    """
    if pointer_angle > max_angle > min_angle:
        # Case (i)
        val_angle = 2 * math.pi - (pointer_angle - min_angle)
        angle_range = 2 * math.pi - (max_angle - min_angle)
        diff_1 = abs(2 * math.pi - (pointer_angle - min_angle))
        diff_2 = abs(pointer_angle - max_angle)
        print(f'get_values_1: case 1')
    elif max_angle > min_angle > pointer_angle:
        # Case (ii) -- covers duplicate branch
        val_angle = min_angle - pointer_angle
        angle_range = 2 * math.pi - (max_angle - min_angle)
        diff_1 = abs(min_angle - pointer_angle)
        diff_2 = abs(2 * math.pi - (max_angle - pointer_angle))
        print(f'get_values_1: case 2')
    elif min_angle > pointer_angle > max_angle:
        # Case (ii) -- covers duplicate branch
        val_angle = min_angle - pointer_angle
        angle_range = min_angle - max_angle
        diff_1 = abs(min_angle - pointer_angle)
        diff_2 = abs(pointer_angle - max_angle)
        print(f'get_values_1: case 3')

    else:
        # If none of the above conditions hold, return None or raise an error.
        angle_range = None
        val_angle = None
        print('get_values_1: Undefined condition')
    
    if angle_range is not None and val_angle is not None:
        print(f'get_values_1: angle_range = {math.degrees(angle_range)} deg, val_angle = {math.degrees(val_angle)} deg')
        print(f'get_values_1: diff_1 = {math.degrees(diff_1)} deg, diff_2 = {math.degrees(diff_2)} deg')
    else:
        print("get_values_2: Angle computations are undefined.")
    
    return angle_range, val_angle, diff_1, diff_2


def get_values_2(max_angle, min_angle, pointer_angle, d_1_angle, d_2_angle):
    """
    Determines the effective angular range and pointer's relative angle (val_angle)
    for a gauge scale when d1_angle is equal to min_angle but d2_angle is not equal to max_angle,
    or when specific conditions for extrapolation apply.
    Returns also diff_1 and diff_2 as measures of the differences.
    """
    # Note: we assume all angles are in radians and normalized.
    if (pointer_angle > min_angle > d_2_angle):
        # Case 1: pointer > min > d2
        val_angle = 2 * math.pi - (pointer_angle - min_angle)
        angle_range = min_angle - d_2_angle
        diff_1 = abs(2 * math.pi - (pointer_angle - min_angle))
        diff_2 = abs(2 * math.pi - (pointer_angle - d_2_angle))
        print('get_values_2: case 1')
    elif (min_angle > pointer_angle > d_2_angle):
        # Case 2: min > pointer > d2
        val_angle = min_angle - pointer_angle
        angle_range = min_angle - d_2_angle
        diff_1 = abs(min_angle - pointer_angle)
        diff_2 = abs(pointer_angle - d_2_angle)
        print('get_values_2: case 2')
    elif (d_2_angle > pointer_angle > min_angle):
        # Case 3: d2 > pointer > min
        val_angle = 2 * math.pi - (pointer_angle - min_angle)
        angle_range = 2 * math.pi - (d_2_angle - min_angle)
        diff_1 = abs(2 * math.pi - (pointer_angle - min_angle))
        diff_2 = abs(d_2_angle - pointer_angle)
        print('get_values_2: case 3')
    elif (d_2_angle > min_angle > pointer_angle):
        # Case 4: d2 > min > pointer
        val_angle = min_angle - pointer_angle
        angle_range = 2 * math.pi - (d_2_angle - min_angle)
        diff_1 = abs(min_angle - pointer_angle)
        diff_2 = abs(2 * math.pi - (d_2_angle - pointer_angle))
        print('get_values_2: case 4')
    
    else:
        val_angle = None
        angle_range = None
        diff_1 = None
        diff_2 = None
        print('get_values_2: Undefined condition')

    if angle_range is not None and val_angle is not None:
        print(f'get_values_2: angle_range = {math.degrees(angle_range)} deg, val_angle = {math.degrees(val_angle)} deg')
        print(f'get_values_2: diff_1 = {math.degrees(diff_1)} deg, diff_2 = {math.degrees(diff_2)} deg')
    else:
        print("get_values_2: Angle computations are undefined.")
    
    return angle_range, val_angle, diff_1, diff_2

    
def get_values_3(max_angle, min_angle, pointer_angle, d_1_angle, d_2_angle):
    """
    Determines the effective angular range and pointer's relative angle (val_angle)
    for a gauge scale when d1_angle is equal to min_angle but d2_angle is not equal to max_angle,
    or when specific conditions for extrapolation apply.
    Returns also diff_1 and diff_2 as measures of the differences.
    """
    # Note: we assume all angles are in radians and normalized.
    if (max_angle > pointer_angle > d_1_angle):
        val_angle = pointer_angle - d_1_angle
        angle_range = 2 * math.pi - (max_angle - d_1_angle)
        diff_1 = abs(pointer_angle - d_1_angle)
        diff_2 = abs(2 * math.pi - (max_angle - pointer_angle))
        print('get_values_3: case 5')
    elif (pointer_angle > max_angle > d_1_angle):
        val_angle = 2 * math.pi - (pointer_angle - d_1_angle)
        angle_range = 2 * math.pi - (max_angle - d_1_angle)
        diff_1 = abs(2 * math.pi - (pointer_angle - d_1_angle))
        diff_2 = abs(pointer_angle - max_angle)
        print('get_values_3: case 6')
    elif (pointer_angle > d_1_angle > max_angle):
        val_angle = pointer_angle - d_1_angle
        angle_range = d_1_angle - max_angle
        diff_1 = abs(pointer_angle - d_1_angle)
        diff_2 = abs(pointer_angle - max_angle)
        print('get_values_3: case 7')
    elif (d_1_angle > max_angle > pointer_angle):
        val_angle = 2 * math.pi - (d_1_angle - pointer_angle)
        angle_range = d_1_angle - max_angle
        diff_1 = abs(2 * math.pi - (d_1_angle - pointer_angle))
        diff_2 = abs(2 * math.pi - (max_angle - pointer_angle))
        print('get_values_3: case 8')
    else:
        val_angle = None
        angle_range = None
        diff_1 = None
        diff_2 = None
        print('get_values_3: Undefined condition')
    
    if angle_range is not None and val_angle is not None:
        print(f'get_values_3: angle_range = {math.degrees(angle_range)} deg, val_angle = {math.degrees(val_angle)} deg')
        print(f'get_values_3: diff_1 = {math.degrees(diff_1)} deg, diff_2 = {math.degrees(diff_2)} deg')
    else:
        print("get_values_3: Angle computations are undefined.")
    
    return angle_range, val_angle, diff_1, diff_2


def get_values_4(max_angle, min_angle, pointer_angle, d1_angle, d2_angle):

    print(f"Passed d_1_angle = {math.degrees(d1_angle) if d1_angle is not None else 'N/A'}, "
      f"d_2_angle = {math.degrees(d2_angle) if d2_angle is not None else 'N/A'}, "
      f"pointer_angle = {math.degrees(pointer_angle) if pointer_angle is not None else 'N/A'}, "
      f"max_angle = {math.degrees(max_angle) if max_angle is not None else 'N/A'}, "
      f"min_angle = {math.degrees(min_angle) if min_angle is not None else 'N/A'} to get_values_1()")

    """
    Given:
      - max_angle: angle (in radians) of the maximum reading (or None)
      - min_angle: angle (in radians) of the minimum reading (or None)
      - pointer_angle: angle (in radians) of the needle pointer.
      - d1_angle: angle (in radians) of the first major division (d1).
      - d2_angle: angle (in radians) of the second major division (d2).
      
    The function determines which sector the pointer lies in based on
    the provided major divisions and either max_angle or min_angle.
    
    It returns:
      angle_range: the effective angular range of the scale (radians)
      value_angle: the pointer's relative angle within that range (radians)
    
    The logic follows the cases you described:
      (a) if max_angle is not None, then various sub-cases (i) to (vi) are applied.
      (b) if min_angle is not None, then another set of sub-cases (i) to (vi) are applied.
    """
    
    # CASE SET (a): Using max_angle if available.
    if max_angle is not None:
        if (d1_angle < max_angle < d2_angle) and (max_angle > pointer_angle):
            # (a)(i)
            angle_range = 2 * math.pi - (d2_angle - d1_angle)
            value_angle = d1_angle - pointer_angle
            diff_1 = abs(d1_angle - pointer_angle)
            diff_2 = abs(2 * math.pi - (d2_angle - pointer_angle))
            print('get_values_2: case (a)(i)')
        elif (d1_angle < max_angle < d2_angle) and (max_angle < pointer_angle):
            # (a)(ii)
            angle_range = 2 * math.pi - (d2_angle - d1_angle)
            value_angle = 2 * math.pi - (pointer_angle - d1_angle)
            diff_1 = abs(2 * math.pi - (pointer_angle - d1_angle))
            diff_2 = abs(d2_angle - pointer_angle)
            print('get_values_2: case (a)(ii)')
        elif (max_angle < d2_angle < d1_angle) and (max_angle < pointer_angle):
            # (a)(iii)
            angle_range = d1_angle - d2_angle
            value_angle = d1_angle - pointer_angle
            diff_1 = abs(d1_angle - pointer_angle)
            diff_2 = abs(d2_angle - pointer_angle)
            print('get_values_2: case (a)(iii)')
        elif (max_angle < d2_angle < d1_angle) and (max_angle > pointer_angle):
            # (a)(iv)
            angle_range = d1_angle - d2_angle
            value_angle = (d1_angle - pointer_angle) - 2 * math.pi
            diff_1 = abs((d1_angle - pointer_angle) - 2 * math.pi)
            diff_2 = abs((d2_angle - pointer_angle) - 2 * math.pi)
            print('get_values_2: case (a)(iv)')
        elif (d2_angle < d1_angle < max_angle) and (max_angle < pointer_angle):
            # (a)(v)
            angle_range = d1_angle - d2_angle
            value_angle = 2 * math.pi - (pointer_angle - d1_angle)
            diff_1 = abs(2 * math.pi - (pointer_angle - d1_angle))
            diff_2 = abs(2 * math.pi - (pointer_angle - d2_angle))
            print('get_values_2: case (a)(v)')
        elif (d2_angle < d1_angle < max_angle) and (max_angle > pointer_angle):
            # (a)(vi)
            angle_range = d1_angle - d2_angle
            value_angle = d1_angle - pointer_angle
            diff_1 = abs(d1_angle - pointer_angle)
            diff_2 = abs(d2_angle - pointer_angle)
            print('get_values_2: case (a)(vi)')
        else:
            print('get_values_2: case (a)(vii): Undefined condition for max_angle')
            angle_range = None
            value_angle = None
            diff_1 = None
            diff_2 = None

    # CASE SET (b): Using min_angle if max_angle is not available.
    elif min_angle is not None:
        if (d1_angle < min_angle < d2_angle) and (min_angle > pointer_angle):
            # (b)(i)
            angle_range = 2 * math.pi - (d2_angle - d1_angle)
            value_angle = d1_angle - pointer_angle
            diff_1 = abs(d1_angle - pointer_angle)
            diff_2 = abs(2 * math.pi - (d2_angle - pointer_angle))
            print('get_values_2: case (b)(i)')
        elif (d1_angle < min_angle < d2_angle) and (min_angle < pointer_angle):
            # (b)(ii)
            angle_range = 2 * math.pi - (d2_angle - d1_angle)
            value_angle = 2 * math.pi - (pointer_angle - d1_angle)
            diff_1 = abs(2 * math.pi - (pointer_angle - d1_angle))
            diff_2 = abs(d2_angle - pointer_angle)
            print('get_values_2: case (b)(ii)')
        elif (d2_angle < d1_angle < min_angle) and (min_angle > pointer_angle):
            # (b)(iii)
            angle_range = d1_angle - d2_angle
            value_angle = d1_angle - pointer_angle
            diff_1 = abs(d1_angle - pointer_angle)
            diff_2 = abs(d2_angle - pointer_angle)
            print('get_values_2: case (b)(iii)')
        elif (d2_angle < d1_angle < min_angle) and (min_angle < pointer_angle):
            # (b)(iv)
            angle_range = d1_angle - d2_angle
            value_angle = 2 * math.pi - (pointer_angle - d1_angle)
            diff_1 = abs(2 * math.pi - (pointer_angle - d1_angle))
            diff_2 = abs(2 * math.pi - (pointer_angle - d2_angle))
            print('get_values_2: case (b)(iv)')
        elif (min_angle < d2_angle < d1_angle) and (min_angle > pointer_angle):
            # (b)(v)
            angle_range = d1_angle - d2_angle
            value_angle = (d1_angle - pointer_angle) - 2 * math.pi
            diff_1 = abs((d1_angle - pointer_angle) - 2 * math.pi)
            diff_2 = abs((d2_angle - pointer_angle) - 2 * math.pi)
            print('get_values_2: case (b)(v)')
        elif (min_angle < d2_angle < d1_angle) and (min_angle < pointer_angle):
            # (b)(vi)
            angle_range = d1_angle - d2_angle
            value_angle = d1_angle - pointer_angle
            diff_1 = abs(d1_angle - pointer_angle)
            diff_2 = abs(d2_angle - pointer_angle)
            print('get_values_2: Case (b)(vi)')
        else:
            print('get_values_2: case (b)(vii): Undefined condition for min_angle')
            angle_range = None
            value_angle = None
            diff_1 = None
            diff_2 = None
    else:
        print("get_values_2: Neither max_angle nor min_angle provided.")
        angle_range = None
        value_angle = None
        diff_1 = None
        diff_2 = None

    if angle_range is not None and value_angle is not None:
        print(f'angle_range = {math.degrees(angle_range)} degrees')
        print(f'value_angle = {math.degrees(value_angle)} degrees')
        print(f'difference_between_d_1_and_pointer = {math.degrees(diff_1)} degrees')
        print(f'difference_between_d_2_and_pointer = {math.degrees(diff_2)} degrees')
    else:
        print("Angle computations are undefined.")

    print("\n")
    return angle_range, value_angle, diff_1, diff_2






def process_frame(frame, frame_id, segmentation_model, detection_model, reader, ocr):

    # final_value = None
    inner_reading = None
    outer_reading = None

    height, width = frame.shape[:2]

    segmented_image = None

    ellipse = None

    number_info_inner = None
    number_info_outer = None

    number_info_inner_1 = None
    number_info_outer_1 = None

    d_1_in_val = None
    d_2_in_val = None
    
    d_1_out_val = None
    d_2_out_val = None
    

    print(f"Processing results for frame {frame_id}")


    # Open the image
    image = frame

    # Get image dimensions (height, width, channels)
    height, width, channels = image.shape
    
    major_axis_length = height
    minor_axis_length = width

    # Run the YOLO segmentation model on the opened image
    segmentation_results = segmentation_model(image)

    # Ensure segmentation results exist for the image
    if segmentation_results[0].masks is not None:
        # Get masks and class IDs
        # All masks
        masks = segmentation_results[0].masks.data.cpu().numpy()
        # Class IDs for the masks
        class_ids = segmentation_results[0].boxes.cls.cpu().numpy()


        # Check for the 'clock' class mask (replace 74 with the correct class ID for 'clock')
        clock_class_id = 74
        clock_mask = None
        for i in range(len(class_ids)):
            class_id = class_ids[i]
            if int(class_id) == clock_class_id:
                clock_mask = masks[i]
                break

        if clock_mask is not None:
            # Process the clock mask
            clock_mask = (clock_mask * 255).astype(np.uint8)

            # Resize mask to match the original image size
            # height, width = image.shape[:2]
            mask_resized = cv2.resize(clock_mask, (width, height), interpolation=cv2.INTER_LINEAR)

            # Clean the mask using morphological operations
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
            cleaned_mask = cv2.morphologyEx(mask_resized, cv2.MORPH_CLOSE, kernel)
            cleaned_mask = cv2.morphologyEx(cleaned_mask, cv2.MORPH_OPEN, kernel)

            # Find contours in the cleaned mask
            contours, _ = cv2.findContours(cleaned_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            valid_contours = [c for c in contours if cv2.contourArea(c) > 500]

            if valid_contours:
                # Get the largest contour
                largest_contour = max(valid_contours, key=cv2.contourArea)

                # Fit an ellipse to the largest contour
                ellipse = cv2.fitEllipse(largest_contour)

                # Extract major and minor axes and angle
                center, axes, angle = ellipse
                major_axis_length = max(axes)
                minor_axis_length = min(axes)

                # Draw the fitted ellipse on the original image
                segmented_image = np.zeros_like(image)  # Black background
                # Apply mask
                segmented_image[cleaned_mask > 0] = image[cleaned_mask > 0]
                

                cv2.ellipse(segmented_image, ellipse, (0, 255, 0), 2)


                results_1 = detection_model(segmented_image)
            else:
                print(f"No valid contours found for 'clock' in: {image}")
                results_1 = detection_model(segmented_image)
        else:
            print(f"No 'clock' mask found in: {image}")
            results_1 = detection_model(image)
    else:
        print(f"No segmentation results for: {image}")
        results_1 = detection_model(image)
    

    if results_1 is None:
        return


    # annotated_image = None


    for result in results_1:
        # Extract bounding boxes and confidence scores
        # Bounding boxes (x_min, y_min, x_max, y_max)
        boxes = result.boxes.xyxy.cpu().numpy()
        # Confidence scores
        confs = result.boxes.conf.cpu().numpy()
        # Class labels
        labels = result.boxes.cls.cpu().numpy()

        # Dictionary to store the highest-confidence box center for each label
        label_centers = {}

        # for i, label in enumerate(labels):
        for i in range(len(labels)):
            label=int(labels[i])
            # x-center of the box
            center_x = (boxes[i][0] + boxes[i][2]) / 2
            # y-center of the box
            center_y = (boxes[i][1] + boxes[i][3]) / 2
            confidence = confs[i]

            # Update the label's center if it has a higher confidence score
            if (label not in label_centers) or (confidence > label_centers[label]['confidence']):
                label_centers[label] = {'center': (center_x, center_y), 'confidence': confidence}

        # Print or save the centers for this image
        print(f"Label centers for {image}:")
        for label in label_centers:
            data = label_centers[label]
            print(f"  Label: {label}, Center: {data['center']}, Confidence: {data['confidence']}")

        # Visualize centers on the image
        # Use YOLO's built-in plotting
        # result_plotted = result.plot(labels=True)

        

        for data in label_centers.values():
            center_x, center_y = data['center']

            # Draw the center as a small circle on the image (modify result_plotted)
            # cv2.circle(result_plotted, (int(center_x), int(center_y)), radius=5, color=(0, 255, 0), thickness=-1)
            if segmented_image is not None:
                result_plotted = cv2.circle(segmented_image, (int(center_x), int(center_y)), radius=5, color=(0, 255, 0), thickness=-1)
            else:
                result_plotted = cv2.circle(image, (int(center_x), int(center_y)), radius=5, color=(0, 255, 0), thickness=-1)

        # label_names=["base", "maximum", "minimum", "tip"]
        
        # Safely extract the center coordinates for each label
        # Label 0
        pointer_base_1 = label_centers.get(0, {}).get('center', None)
        # Label 1
        maximum_1 = label_centers.get(1, {}).get('center', None)
        # Label 2
        minimum_1 = label_centers.get(2, {}).get('center', None)
        # Label 3   
        pointer_tip_1 = label_centers.get(3, {}).get('center', None)


        pointer_base = None
        pointer_tip = None
        maximum = None
        minimum = None


        if pointer_base_1 is not None :
            pointer_base = [pointer_base_1[0], height - pointer_base_1[1]]
        else :
            continue

        if pointer_tip_1 is not None :
            pointer_tip = [pointer_tip_1[0], height - pointer_tip_1[1]]
        else :
            continue

        if maximum_1 is not None :
            maximum = [maximum_1[0], height - maximum_1[1]]
        else :
            continue

        if minimum_1 is not None :
            minimum = [minimum_1[0], height - minimum_1[1]]
        else :
            continue


        # Print the extracted values
        print(f"Pointer Base: {pointer_base}")
        print(f"Maximum: {maximum}")
        print(f"Minimum: {minimum}")
        print(f"Pointer Tip: {pointer_tip}")
        print("\n")


        if image is None:
            print(f"Failed to read {frame_id}")
            continue

        if (pointer_base is not None) and (minimum is not None) and (maximum is not None) and (ellipse is not None):
            height = image.shape[0]

            # Convert to Cartesian
            cx, cy = pointer_base_1
            cy_c = height - cy  # Cartesian
            max_x, max_y = maximum_1
            max_y_c = height - max_y
            min_x, min_y = minimum_1
            min_y_c = height - min_y

            a_0 = major_axis_length / 2
            b_0 = minor_axis_length / 2
            theta = np.deg2rad(-angle)  # NEGATE angle for Cartesian rotation

            # Transform max point to aligned frame
            X1, Y1 = rotate_point_1(max_x, max_y_c, cx, cy_c, theta)
            X2, Y2 = rotate_point_1(min_x, min_y_c, cx, cy_c, theta)

            # Get scaled ellipse
            a_1, b_1 = get_scaled_axes(X1, Y1, a_0, b_0)
            a_2, b_2 = get_scaled_axes(X2, Y2, a_0, b_0)

            # Prepare image to draw
            output_img = image.copy()

            # Draw original ellipse from fit (OpenCV format uses image coordinates)
            cv2.ellipse(output_img, ellipse, (0, 0, 255), 2)  # Red


            # Compute alpha: angle of (X1, Y1) vector in rotated (ellipse-aligned) frame
            alpha_1 = np.arctan2(Y1, X1)  # In radians
            alpha_2 = np.arctan2(Y2, X2)  # In radians

            # Check direction of alpha: whether it's closer to major or minor axis
            # Since in rotated frame, major axis is along X and minor along Y

            if abs(np.cos(alpha_1)) < abs(np.sin(alpha_1)):
                # Closer to Y-axis ⇒ longer component along minor axis ⇒ swap needed
                a_1, b_1 = b_1, a_1
                angle_for_cv2_1 = (angle + 90) % 180
            else:
                angle_for_cv2_1 = angle

            if abs(np.cos(alpha_2)) < abs(np.sin(alpha_2)):
                # Closer to Y-axis ⇒ longer component along minor axis ⇒ swap needed
                a_2, b_2 = b_2, a_2
                angle_for_cv2_2 = (angle + 90) % 180
            else:
                angle_for_cv2_2 = angle


            ellipse_1 = (pointer_base_1, (2 * a_1, 2 * b_1), angle)
            # cv2.ellipse(output_img, ellipse_1, (0, 255, 0), 2)  # Green

            ellipse_2 = (pointer_base_1, (2 * a_2, 2 * b_2), angle)
            # cv2.ellipse(output_img, ellipse_2, (255, 0, 0), 2)  # Blue
            

            # a_3 = max(a_1,a_2)
            # b_3 = max(b_1,b_2)

            # a_4 = min(a_1,a_2)
            # b_4 = min(b_1,b_2)

            a_3 = (a_1 + a_2) / 2
            b_3 = (b_1 + b_2) / 2

            a_4 = a_3
            b_4 = b_3

            ellipse_3 = (pointer_base_1, (2 * a_3, 2 * b_3), angle)
            cv2.ellipse(output_img, ellipse_3, (255, 0, 255), 2)  # Purple
            cv2.ellipse(result_plotted, ellipse_3, (255, 0, 255), 2)  # Purple

            ellipse_4 = (pointer_base_1, (2 * a_3 * 0.5, 2 * b_3 * 0.5), angle)
            cv2.ellipse(output_img, ellipse_4, (255, 255, 0), 2)  # Light-Blue
            cv2.ellipse(result_plotted, ellipse_4, (255, 255, 0), 2)  # Light-Blue

            # # Save image
            # save_path_1 = os.path.join(output_folder_1, os.path.basename(image_path))
            # cv2.imwrite(save_path_1, output_img)
            # print(f"Saved: {save_path_1}")

            
            # annotated_image, num_info = detect_and_draw_boxes(result_plotted, ocr)

            # Extract ring-shaped region
            inner_segmented_ring = extract_elliptical_ring(image.copy(), pointer_base_1, (int(2 * a_3), int(2 * b_3)), (int(2 * a_3 * 0.5), int(2 * b_3 * 0.5)), angle)

            # Get annotated image and recognized integers
            number_info_inner_1 = detect_and_draw_boxes(inner_segmented_ring.copy(), ocr)

            number_info_inner = []
            for i in range(len(number_info_inner_1)):
                x, y = number_info_inner_1[i][0]
                value = number_info_inner_1[i][1]
                number_info_inner.append(((x, height - y), value))


            # number_info would look like: [((x1, y1), 12), ((x2, y2), 60), ...]
            print("Recognized inner numeric labels:", number_info_inner)

            # Extract ring-shaped region
            outer_segmented_ring = extract_elliptical_ring(image.copy(), pointer_base_1, (int(2 * a_4 * 1.5), int(2 * b_4 * 1.5)), (int(2 * a_4), int(2 * b_4)), angle)
            # Get annotated image and recognized integers
            number_info_outer_1 = detect_and_draw_boxes(outer_segmented_ring.copy(), ocr)
            # number_info would look like: [((x1, y1), 12), ((x2, y2), 60), ...]

            number_info_outer = []
            for i in range(len(number_info_outer_1)):
                x, y = number_info_outer_1[i][0]
                value = number_info_outer_1[i][1]
                number_info_outer.append(((x, height - y), value))

            # number_info would look like: [((x1, y1), 12), ((x2, y2), 60), ...]
            print("Recognized outer numeric labels:", number_info_outer)





        needle_length = major_axis_length / 2

        # Example processing of the gauge value
        if (minimum is not None) and \
            (maximum is not None) and \
                (pointer_base is not None) and \
                    (pointer_tip is not None):
                
                needle_length = math.sqrt(((pointer_base[0] - pointer_tip[0]) ** 2) + ((pointer_base[1] - pointer_tip[1]) ** 2))

        if major_axis_length is None:
            major_axis_length = 3 * needle_length
        
        if minor_axis_length is None:
            minor_axis_length = 3 * needle_length
        
        # if minimum_1 is not None:
        #     cv2.circle(result_plotted, (int(minimum_1[0]), int(minimum_1[1])), radius=int(major_axis_length/10), color=(0, 0, 255), thickness=2)
        #     min_val=ocr_detect(reader, ocr, 0, minimum_1[0], minimum_1[1], major_axis_length/9, image.copy())
        #     print(f"Minimum Value = {min_val}")
        # print("\n")
        
        # if maximum_1 is not None:
        #     cv2.circle(result_plotted, (int(maximum_1[0]), int(maximum_1[1])), radius=int(major_axis_length/10), color=(0, 0, 255), thickness=2)
        #     max_val=ocr_detect(reader, ocr, 1, maximum_1[0], maximum_1[1], major_axis_length/9, image.copy())
        #     print(f"Maximum Value = {max_val}")
        # print("\n")



    # output_path_1 = os.path.join(output_folder_11, f"result_{os.path.basename(image_path)}")
    # Save the transformed image
    # cv2.imwrite(output_path_1, result_plotted)
    # print(f"Results saved to: {output_path_1}")
    print("\n")


    ellipse_inclination = None

    if ellipse is not None :
        ellipse_inclination = angle

    # final_value = None

    inner_reading = None
    outer_reading = None

    d_1_in = None
    d_2_in = None
                
    d_1_out = None
    d_2_out = None

    # inner_div_1 = None
    # inner_div_2 = None

    # outer_div_1 = None
    # outer_div_2 = None
    
    if (pointer_base is not None) and \
        (pointer_tip is not None) and \
            (ellipse is not None):
        
        # Get the angles for minimum, maximum, and pointer positions
        min_angle = get_angle(pointer_base, minimum, major_axis_length, minor_axis_length, ellipse_inclination)
        max_angle = get_angle(pointer_base, maximum, major_axis_length, minor_axis_length, ellipse_inclination)
        pointer_angle = get_angle(pointer_base, pointer_tip, major_axis_length, minor_axis_length, ellipse_inclination)
        

        # # If all the angles are available, perform linear interpolation with corrected angles
        # if (max_angle is not None) and \
        #     (min_angle is not None) and \
        #     (pointer_angle is not None):
            
        #     # print(f'min_angle = {math.degrees(min_angle)}')
        #     # print(f"min_val = {min_val}")
        #     # print("\n")
        #     # print(f'max_angle = {math.degrees(max_angle)}')
        #     # print(f"max_val = {max_val}")
        #     # print("\n")
        #     # print(f'pointer_angle = {math.degrees(pointer_angle)}')
            
        #     angle_range, val_angle = get_values_11(max_angle, min_angle, pointer_angle)
            
        #     if (max_val is not None) and \
        #         (min_val is not None) and \
        #         (angle_range is not None) and\
        #         (val_angle is not None):

        #         final_value_1 = min_val + (((max_val - min_val) * val_angle) / angle_range)
                
                # good_images[0].add(image_path)
                # good_images_val[0][image_path] = final_value_1
                # print(f'final_value_1 = {final_value_1}')
                # print("\n")

                # if inner_readings[image_path] is not None :
                #     original_value = inner_readings[image_path]
                # elif outer_readings[image_path] is not None :
                #     original_value = outer_readings[image_path]
                # else:
                #     continue

                # abs_error = abs(final_value_1 - original_value)
                # error = 0
                # if original_value > 0 :
                #     error = (100 * abs_error) / abs(original_value)
                # print(f"{image_path} :- ")
                # print(f"final_value_1")
                # print(f"Detected Value = {final_value_1}")
                # print(f"Original Value = {original_value}")
                # print(f"% Error = {error}")
                # print(f"Absolute Error = {abs_error}")
                # print("\n")


        
        
        # if (min_angle is not None) and (min_val is not None) :
        #     print(f'min_angle = {math.degrees(min_angle)}')
        #     print(f"min_val = {min_val}")

        # print("\n")

        # if (max_angle is not None) and (max_val is not None) :
        #     print(f'max_angle = {math.degrees(max_angle)}')
        #     print(f"max_val = {max_val}")

        # print("\n")

        print("\n")
        print(f"inner_reading calculations :-")
        print("\n")
        
        inner_division_angles = []
        inner_division_values = []
        
        if number_info_inner is not None:
            for i in range(len(number_info_inner)):
                inner_division_values.append(number_info_inner[i][1])
                inner_division_angles.append(None)
                inner_division_angles[i] = get_angle(pointer_base, number_info_inner[i][0], major_axis_length, minor_axis_length, ellipse_inclination)
        else:
            return


        if (inner_division_values is not None) and \
            (inner_division_angles is not None) :


            print(f'pointer_angle = {math.degrees(pointer_angle)}')
            print("\n")

            for g in range(len(inner_division_angles)):
                if inner_division_angles[g] is not None:
                    print(f'Inner Division {g} Angle = {math.degrees(inner_division_angles[g])}')
                    print(f'Inner Division {g} Value = {inner_division_values[g]}')  # <-- This line fixed
                else:
                    print(f"Inner Division {g} Angles = N/A")
                    print(f"Inner Division {g} Value = N/A")


                print("\n")
            

            # diff_sum = val_angle + (angle_range - val_angle)
            diff_sum = 1e9

            for a in range(len(inner_division_angles)) :
                for b in range(len(inner_division_angles)) :
                    if (a != b) and \
                        (inner_division_values[a] is not None) and \
                            (inner_division_values[b] is not None) and \
                                (inner_division_values[a] < inner_division_values[b]):
                                
                        inner_division_angle_range_1, inner_division_val_angle_1, diff_1_1, diff_2_1 = get_values(max_angle, min_angle, pointer_angle, inner_division_angles[a], inner_division_angles[b], inner_division_values[a], inner_division_values[b])
                        
                        if (diff_sum is None) or \
                                (diff_1_1 is not None) or \
                                    (diff_2_1 is not None) :
                            if (diff_1_1 + diff_2_1 < diff_sum) :
                                d_1_in = a
                                d_2_in = b
                                diff_sum = diff_1_1 + diff_2_1
                                inner_division_angle_range = inner_division_angle_range_1
                                inner_division_val_angle = inner_division_val_angle_1
                                # diff_1 = diff_1_1
                                # diff_2 = diff_2_1

                        else :
                            print("Difference angles are not available")
            
            # inner_div_1 = inner_division_values[d_1_in] if d_1_in is not None else None
            # inner_div_2 = inner_division_values[d_2_in] if d_2_in is not None else None
            
            # if (annotated_image is not None) and (d_1_in is not None):
            #     cv2.circle(annotated_image, number_info_inner_1[d_1_in][0], radius=5, color=(0, 0, 255), thickness=-1) # red
            
            # if (annotated_image is not None) and (d_2_in is not None):
            #     cv2.circle(annotated_image, number_info_inner_1[d_2_in][0], radius=5, color=(0, 0, 255), thickness=-1) # red
            
            print("\n")
            print(f"Finally chosen d_1_angle = {math.degrees(inner_division_angles[d_1_in]) if d_1_in is not None else 'N/A'}, d_1_val = {inner_division_values[d_1_in] if d_1_in is not None else 'N/A'} and d_2_angle = {math.degrees(inner_division_angles[d_2_in]) if d_2_in is not None else 'N/A'}, d_2_val = {inner_division_values[d_2_in] if d_2_in is not None else 'N/A'} for inner_reading calculations")
            print("\n")

            if (d_1_in is not None) and (d_2_in is not None) :
                inner_reading = inner_division_values[d_1_in] + (((inner_division_values[d_2_in] - inner_division_values[d_1_in]) * inner_division_val_angle) / inner_division_angle_range)

                # good_images[2].add(image_path)
                # good_images_val[2][image_path] = inner_reading
                # print(f'inner_reading = {inner_reading}')
                # print("\n")

                # if inner_readings[image_path] is not None :
                #     abs_error = abs(inner_reading - inner_readings[image_path])
                #     error = 0
                #     if inner_readings[image_path] > 0 :
                #         error = (100 * abs_error) / abs(inner_readings[image_path])
                #     print(f"{image_path} :- ")
                #     print(f"inner_reading")
                #     print(f"Detected Value = {inner_reading}")
                #     print(f"Original Value = {inner_readings[image_path]}")
                #     print(f"% Error = {error}")
                #     print(f"Absolute Error = {abs_error}")
                #     print("\n")
                # else:
                #     continue


        print("\n")
        print(f"outer_reading calculations :-")
        print("\n")

        outer_division_angles = []
        outer_division_values = []

        if number_info_outer is not None:
            for i in range(len(number_info_outer)):
                outer_division_values.append(number_info_outer[i][1])
                outer_division_angles.append(None)
                outer_division_angles[i] = get_angle(pointer_base, number_info_outer[i][0], major_axis_length, minor_axis_length, ellipse_inclination)
        else:
            return



        if (outer_division_values is not None) and \
            (outer_division_angles is not None) :


            print(f'pointer_angle = {math.degrees(pointer_angle)}')
            print("\n")

            for g in range(len(outer_division_angles)):
                if outer_division_angles[g] is not None:
                    print(f'Outer Division {g} Angle = {math.degrees(outer_division_angles[g])}')
                    print(f'Outer Division {g} Value = {outer_division_values[g]}')
                else:
                    print(f"Outer Division {g} Angle = N/A")
                    print(f"Outer Division {g} Value = N/A")

                print("\n")


            # diff_sum = val_angle + (angle_range - val_angle)
            diff_sum = 1e9

            for a in range(len(outer_division_angles)) :
                for b in range(len(outer_division_angles)) :
                    if (a != b) and \
                        (outer_division_values[a] is not None) and \
                            (outer_division_values[b] is not None) and \
                                (outer_division_values[a] < outer_division_values[b]):
                                
                        outer_division_angle_range_1, outer_division_val_angle_1, diff_1_1, diff_2_1 = get_values(max_angle, min_angle, pointer_angle, outer_division_angles[a], outer_division_angles[b], outer_division_values[a], outer_division_values[b])
                        
                        if (diff_sum is None) or \
                                (diff_1_1 is not None) or \
                                    (diff_2_1 is not None) :
                            if (diff_1_1 + diff_2_1 < diff_sum) :
                                d_1_out = a
                                d_2_out = b
                                diff_sum = diff_1_1 + diff_2_1
                                outer_division_angle_range = outer_division_angle_range_1
                                outer_division_val_angle = outer_division_val_angle_1
                                # diff_1 = diff_1_1
                                # diff_2 = diff_2_1

                        else :
                            print("Difference angles are not available")
            
            # outer_div_1 = outer_division_values[d_1_out] if d_1_out is not None else None
            # outer_div_2 = outer_division_values[d_2_out] if d_2_out is not None else None

            # if (annotated_image is not None) and (d_1_out is not None):
            #     cv2.circle(annotated_image, number_info_outer_1[d_1_out][0], radius=5, color=(0, 0, 255), thickness=-1) # red
            
            # if (annotated_image is not None) and (d_2_out is not None):
            #     cv2.circle(annotated_image, number_info_outer_1[d_2_out][0], radius=5, color=(0, 0, 255), thickness=-1) # red

            print("\n")
            print(f"Finally chosen d_1_angle = {math.degrees(outer_division_angles[d_1_out]) if d_1_out is not None else 'N/A'}, d_1_val = {outer_division_values[d_1_out] if d_1_out is not None else 'N/A'} and d_2_angle = {math.degrees(outer_division_angles[d_2_out]) if d_2_out is not None else 'N/A'}, d_2_val = {outer_division_values[d_2_out] if d_2_out is not None else 'N/A'} for outer_reading calculations")
            print("\n")

            if (d_1_out is not None) and (d_2_out is not None) :
                outer_reading = outer_division_values[d_1_out] + (((outer_division_values[d_2_out] - outer_division_values[d_1_out]) * outer_division_val_angle) / outer_division_angle_range)

                # good_images[3].add(image_path)
                # good_images_val[3][image_path] = outer_reading
                # print(f'outer_reading = {outer_reading}')
                # print("\n")

                # if outer_readings[image_path] is not None :
                #     abs_error = abs(outer_reading - outer_readings[image_path])
                #     error = 0
                #     if outer_readings[image_path] > 0 :
                #         error = (100 * abs_error) / abs(outer_readings[image_path])
                #     print(f"{image_path} :- ")
                #     print(f"outer_reading")
                #     print(f"Detected Value = {outer_reading}")
                #     print(f"Original Value = {outer_readings[image_path]}")
                #     print(f"% Error = {error}")
                #     print(f"Absolute Error = {abs_error}")
                #     print("\n")
                # else:
                #     continue


    # print(f"Processing completed for: {image_path}")
    print("\n")


    # # Overlay the three readings on the frame
    # overlay_texts = [
    #     # f"Main Gauge: {final_value_1:.2f}" if final_value_1 is not None else "Main Gauge: N/A",
    #     f"Inner Reading: {inner_reading:.2f} using {inner_division_values[d_1]} and {inner_division_values[d_2]}" if inner_reading is not None else "Inner Reading: N/A",
    #     f"Outer Reading: {outer_reading:.2f} using {outer_division_values[d_1]} and {outer_division_values[d_2]}" if outer_reading is not None else "Outer Reading: N/A",
    # ]






    # overlay_texts = [
    #     (
    #         f"Inner Reading: {inner_reading:.2f} using {inner_div_1} and {inner_div_2}"
    #         if inner_reading is not None and inner_div_1 is not None and inner_div_2 is not None
    #         else "Inner Reading: N/A"
    #     ),
    #     (
    #         f"Outer Reading: {outer_reading:.2f} using {outer_div_1} and {outer_div_2}"
    #         if outer_reading is not None and outer_div_1 is not None and outer_div_2 is not None
    #         else "Outer Reading: N/A"
    #     ),
    # ]


    # y0, dy = 30, 30
    # for i, text in enumerate(overlay_texts):
    #     y = y0 + i*dy
    #     cv2.putText(frame, text, (20, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    #     if annotated_image is not None:
    #         cv2.putText(annotated_image, text, (20, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)


    gc.collect()

    # return frame, final_value, inner_reading, outer_reading

    # if annotated_image is not None:
    #     return annotated_image, final_value, inner_reading, outer_reading
    # else:
    #     # return frame, final_value, inner_reading, outer_reading
    #     return final_value, inner_reading, d_1_in, d_2_in, outer_reading, d_1_out, d_2_out


    # if None in [d_1_in, d_2_in, d_1_out, d_2_out]:
    #     print(f"Frame {frame_id}: Skipping due to None in division index")
    #     return None

    # return inner_reading, inner_division_values[d_1_in], inner_division_values[d_2_in], \
    #     outer_reading, outer_division_values[d_1_out], outer_division_values[d_2_out]


    if None in [d_1_in, d_2_in]:
        inner_reading = None
        d_1_in_val = None
        d_2_in_val = None
    else:
        d_1_in_val = inner_division_values[d_1_in]
        d_2_in_val = inner_division_values[d_2_in]

    if None in [d_1_out, d_2_out]:
        outer_reading = None
        d_1_out_val = None
        d_2_out_val = None
    else:
        d_1_out_val = outer_division_values[d_1_out]
        d_2_out_val = outer_division_values[d_2_out]

    return inner_reading, d_1_in_val, d_2_in_val, outer_reading, d_1_out_val, d_2_out_val



def create_reading_display(inner_reading, d_1_in_val, d_2_in_val, outer_reading, d_1_out_val, d_2_out_val):

    width=1000
    height=100

    # Create a blank image (black background)
    display = np.zeros((height, width, 3), dtype=np.uint8)

    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.8
    thickness = 2
    color = (0, 255, 0)

    inner_text = (
        f"Inner Reading: {inner_reading:.2f} using {d_1_in_val} and {d_2_in_val}" if inner_reading is not None else "Inner Reading: N/A"
    )
    outer_text = (
        f"Outer Reading: {outer_reading:.2f} using {d_1_out_val} and {d_2_out_val}" if outer_reading is not None else "Outer Reading: N/A"
    )

    cv2.putText(display, inner_text, (20, 40), font, font_scale, color, thickness)
    cv2.putText(display, outer_text, (20, 80), font, font_scale, color, thickness)

    return display


def main():
    # Load YOLO models
    segmentation_model = YOLO("yolo11n-seg.pt")
    detection_model = YOLO(os.path.join("pretrained_models", "5", "last.pt"))

    # Output folder setup
    output_folder = "result_videos"
    os.makedirs(output_folder, exist_ok=True)
    # output_video_path = os.path.join(output_folder, "final_output.mp4")

    # Initialize OCR
    reader = easyocr.Reader(['en'], gpu=True)
    ocr = PaddleOCR(use_angle_cls=True, lang='en', rec=True, device='cpu')

    # Input video source
    # video_path = "test_videos/195199132_main_xxl.mp4"

    video_no = 9
    # Input video path
    video_path = f"test_videos/video_{video_no}.mp4"
    # Output video path
    output_video_path = os.path.join(output_folder, f"final_video_output_{video_no}.mp4")

    # Open a video file or webcam stream (use 0 for webcam).
    cap = cv2.VideoCapture(video_path)
    # cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print(f"Error opening video: {video_path}")
        return

    # Get video properties
    # fps = cap.get(cv2.CAP_PROP_FPS) or 30
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    if not out.isOpened():
        print("Error: Could not open VideoWriter")
        return

    print("Starting video processing... Press 'q' to quit.")
    frame_id = 0

    while True:
        start_time = time.time()

        ret, frame = cap.read()
        if not ret:
            break
        frame_id += 1

        ans = process_frame(frame, frame_id, segmentation_model, detection_model, reader, ocr)

        if ans is None:
            print(f"Frame {frame_id}: Processing failed or skipped.")
            continue  # or break if you want to stop on failure
        
        inner_reading, d_1_in_val, d_2_in_val, outer_reading, d_1_out_val, d_2_out_val = ans  # ignore processed_frame

        # Show raw frame
        cv2.imshow("Raw Frame", frame)

        # Display readings
        reading_display = create_reading_display(inner_reading, d_1_in_val, d_2_in_val, outer_reading, d_1_out_val, d_2_out_val)
        cv2.imshow("Detected Readings", reading_display)



        
        # if ans is not None :
        #     # Process the current frame
        #     processed_frame, final_value, inner_reading, outer_reading = ans
        # else :
        #     return

        # # Show result
        # cv2.imshow("Gauge Detection", processed_frame)
        # out.write(processed_frame)


        # cv2.imshow("Raw Frame", frame)

        # # Create and show the readings in a separate window
        # reading_display = create_reading_display(inner_reading, outer_reading)
        # cv2.imshow("Detected Readings", reading_display)

        

        # # Calculate how much time we spent on this frame
        # processing_time = time.time() - start_time

        # # Determine how many frames to skip based on processing time
        # frames_to_skip = int(processing_time * fps)
        # for _ in range(frames_to_skip):
        #     cap.grab()  # Skip frame without decoding




        # # Calculate elapsed processing time and determine frames to skip.
        # processing_time = time.time() - start_time
        # frames_to_skip = int(processing_time * fps)
        
        # # Instead of just skipping, duplicate the processed frame for the number of skipped frames + 1.
        # duplicate_count = frames_to_skip + 1
        # for _ in range(duplicate_count):
        #     out.write(processed_frame)
        
        # # Skip frames without processing.
        # for _ in range(frames_to_skip):
        #     cap.grab()  # Grab without decoding.




        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Quit signal received.")
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"Done! Output saved to {output_video_path}")


if __name__ == "__main__":
    main()


[2025/08/25 23:03:48] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\Harsh/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\Harsh/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=

ValueError: too many values to unpack (expected 4)